# Data

In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [2]:
df = pd.read_csv("amazon.csv")
df = df.drop(['product_id', 'product_name', 'category', 'actual_price', 'about_product', 'user_id', 'user_name', 'review_id', 'review_title', 'review_content', 'img_link', 'product_link'], axis=1)

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
df['discount_percentage'] = df['discount_percentage'].astype('str').str.replace('%', '').astype('float64')
df['discount_percentage'] = df['discount_percentage'] / 100

df['rating_count'] = df['rating_count'].astype('str').str.replace(',', '').astype('float64')
df['discounted_price'] = df['discounted_price'].astype('str').str.replace("₹",'')
df['discounted_price'] = df['discounted_price'].str.replace(",",'')
df['discounted_price'] = df['discounted_price'].astype('float64')

df['rating'] = df['rating'].astype('float64')

In [5]:
y = df['rating']
X = df

In [6]:
class RatingDataset (Dataset):
    def __init__(self):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

# Dependencies

In [7]:
from torch import nn
import torch
from torch.utils.data import random_split

# Model

In [8]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [9]:
class RatingPredicter(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(len(X.columns), 64),
            nn.Sigmoid(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.Sigmoid(),
            nn.Linear(16, 1),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.network(x)
        return logits

In [10]:
model = RatingPredicter().to(device)

# Train

In [11]:
learning_rate = 1e-3
batch_size = 64
epochs = 20

In [12]:
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
dataset = RatingDataset()
train_set, test_set = random_split(dataset, [4/5, 1/5])

train_dataloader = DataLoader(train_set, shuffle=False, batch_size=batch_size)
test_dataloader = DataLoader(test_set, shuffle=False)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, opt)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------


KeyError: 437